In [24]:
import pytorch_lightning as pl
import torch 
import torch.nn as nn

from torchmetrics import Accuracy

torch.set_float32_matmul_precision('medium')


In [25]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms


class MnistDataModule(pl.LightningDataModule):
    def __init__(self, data_path='./MNIST/') -> None:
        super().__init__()
        self.data_path = data_path
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def prepare_data(self):
        MNIST(self.data_path, download=True)

    def setup(self, stage: str):

        mnist_all = MNIST(
            root=self.data_path,
            train=True,
            transform=self.transform,
            download=False
        )

        self.train, self.val = random_split(
            mnist_all, [55000, 5000], 
            generator=torch.Generator().manual_seed(1)
        )

        self.test = MNIST(
            root=self.data_path,
            train=False,
            transform=self.transform,
            download=False
        )
    
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=64, num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(self.val, batch_size=64, num_workers=4)
    
    def test_dataloader(self):
        return DataLoader(self.test, batch_size=64, num_workers=4)


In [26]:
from typing import Any


class MultiLayerPerceptron(pl.LightningModule):
    def __init__(self, 
                 image_shape=(1, 28, 28), hidden_units=(32, 16)):
        super().__init__()

        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.valid_acc   = Accuracy(task="multiclass", num_classes=10)
        self.test_acc  = Accuracy(task="multiclass", num_classes=10)

        input_size = image_shape[0] * image_shape[1] * image_shape[2]
        self.fl = nn.Flatten()
        self.l1 = nn.Linear(input_size, 32)
        self.a1 = nn.ReLU()
        self.l2 = nn.Linear(32, 16)
        self.a2 = nn.ReLU()
        self.l3 = nn.Linear(16, 10)
        self.a3 = nn.Softmax()

    def forward(self, x):
        x = self.fl(x)
        x = self.l1(x)
        x = self.a1(x)
        x = self.l2(x)
        x = self.a2(x)
        x = self.l3(x)
        x = self.a3(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(self(x), y)
        preds = torch.argmax(logits, dim=1)
        self.train_acc.update(preds, y)
        self.log("train loss: ",loss, prog_bar=True)
        return loss
    
    def on_train_epoch_end(self):
        self.log("train acc :", self.train_acc.compute())

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(self(x), y)
        preds = torch.argmax(logits, dim=1)
        self.valid_acc.update(preds, y)
        self.log("valid_loss", loss, prog_bar=True)
        self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(self(x), y)
        preds = torch.argmax(logits, dim=1)
        self.test_acc.update(preds, y)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_acc.compute(), prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = 0.001)
        return optimizer



        

In [33]:
torch.manual_seed(1)
mnist_dm = MnistDataModule()
mnistClassifier = MultiLayerPerceptron()

# trainer = pl.Trainer(max_epochs=10, accelerator="auto")

trainer = pl.Trainer(max_epochs=100, accelerator="auto", enable_checkpointing=True)

trainer.fit(model=mnistClassifier, datamodule=mnist_dm
            ,ckpt_path='/home/tej/Documents/Courses/Learning/ML_With_PyTorch_Scikit_Practice/Chapter13/lightning_logs/version_0/checkpoints/epoch=49-step=43000.ckpt')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /home/tej/Documents/Courses/Learning/ML_With_PyTorch_Scikit_Practice/Chapter13/lightning_logs/version_0/checkpoints/epoch=49-step=43000.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | train_acc | MulticlassAccuracy | 0     
1 | valid_acc | MulticlassAccuracy | 0     
2 | test_acc  | MulticlassAccuracy | 0     
3 | fl        | Flatten            | 0     
4 | l1        | Linear             | 25.1 K
5 | a1        | ReLU               | 0     
6 | l2        | Linear             | 528   
7 | a2        | ReLU               | 0     
8 | l3        | Linear             | 170   
9 | a3        | Softmax            | 0     
-------------------------------------------------
25.8 K    Trainable params
0       

Epoch 99: 100%|██████████| 860/860 [00:02<00:00, 315.26it/s, v_num=1, train loss: =1.500, valid_loss=1.500, valid_acc=0.959]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 860/860 [00:02<00:00, 314.73it/s, v_num=1, train loss: =1.500, valid_loss=1.500, valid_acc=0.959]


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir ./Chapter13/lightning_logs/